In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#Custom layers

In [2]:
from keras.layers import Layer, InputSpec

try:
    from keras import initializations
except ImportError:
    from keras import initializers as initializations
import keras.backend as K


class Scale(Layer):
    '''Custom Layer for DenseNet used for BatchNormalization.

    Learns a set of weights and biases used for scaling the input data.
    the output consists simply in an element-wise multiplication of the input
    and a sum of a set of constants:

        out = in * gamma + beta,

    where 'gamma' and 'beta' are the weights and biases larned.

    # Arguments
        axis: integer, axis along which to normalize in mode 0. For instance,
            if your input tensor has shape (samples, channels, rows, cols),
            set axis to 1 to normalize per feature map (channels axis).
        momentum: momentum in the computation of the
            exponential average of the mean and standard deviation
            of the data, for feature-wise normalization.
        weights: Initialization weights.
            List of 2 Numpy arrays, with shapes:
            `[(input_shape,), (input_shape,)]`
        beta_init: name of initialization function for shift parameter
            (see [initializations](../initializations.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
        gamma_init: name of initialization function for scale parameter (see
            [initializations](../initializations.md)), or alternatively,
            Theano/TensorFlow function to use for weights initialization.
            This parameter is only relevant if you don't pass a `weights` argument.
    '''

    def __init__(self, weights=None, axis=-1, momentum=0.9, beta_init='zero', gamma_init='one', **kwargs):
        self.momentum = momentum
        self.axis = axis
        self.beta_init = initializations.get(beta_init)
        self.gamma_init = initializations.get(gamma_init)
        self.initial_weights = weights
        super(Scale, self).__init__(**kwargs)

    def build(self, input_shape):
        self.input_spec = [InputSpec(shape=input_shape)]
        shape = (int(input_shape[self.axis]),)

        # Tensorflow >= 1.0.0 compatibility
        self.gamma = K.variable(self.gamma_init(shape), name='{}_gamma'.format(self.name))
        self.beta = K.variable(self.beta_init(shape), name='{}_beta'.format(self.name))
        # self.gamma = self.gamma_init(shape, name='{}_gamma'.format(self.name))
        # self.beta = self.beta_init(shape, name='{}_beta'.format(self.name))
        self._trainable_weights = [self.gamma, self.beta]

        if self.initial_weights is not None:
            self.set_weights(self.initial_weights)
            del self.initial_weights

    def call(self, x, mask=None):
        input_shape = self.input_spec[0].shape
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]

        out = K.reshape(self.gamma, broadcast_shape) * x + K.reshape(self.beta, broadcast_shape)
        return out

    def get_config(self):
        config = {"momentum": self.momentum, "axis": self.axis}
        base_config = super(Scale, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

#DenseNet

In [3]:
import keras.backend as K
from keras.layers import BatchNormalization
from keras.layers import ZeroPadding2D, Concatenate
from keras.layers.convolutional import Convolution2D
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.pooling import AveragePooling2D, GlobalAveragePooling2D, MaxPooling2D
from keras.models import Model

def DenseNet(img_input_layer, img_n_label_input_layer, label_input_layer, nb_dense_block=4, growth_rate=32, nb_filter=64, reduction=0.0, dropout_rate=0.0, weight_decay=1e-4,
             classes=1000, weights_path=None):
    """Instantiate the DenseNet 121 architecture,
        # Arguments
            nb_dense_block: number of dense blocks to add to end
            growth_rate: number of filters to add per dense block
            nb_filter: initial number of filters
            reduction: reduction factor of transition blocks.
            dropout_rate: dropout rate
            weight_decay: weight decay factor
            classes: optional number of classes to classify images
            weights_path: path to pre-trained weights
        # Returns
            A Keras model instance.
    """
    eps = 1.1e-5

    # compute compression factor
    compression = 1.0 - reduction

    # Handle Dimension Ordering for different backends
    global concat_axis
    if K.image_data_format() == 'channels_last': # Tensorflow
        concat_axis = 3
        # img_input = Input(shape=(224, 224, 3), name='data')
        img_input = img_n_label_input_layer
    else: # Theano
        concat_axis = 1
        # img_input = Input(shape=(3, 224, 224), name='data')
        img_input = img_n_label_input_layer

    # From architecture for ImageNet (Table 1 in the paper)
    nb_filter = 64
    nb_layers = [6, 12, 24, 16]  # For DenseNet-121

    # Initial convolution
    x = ZeroPadding2D((3, 3), name='conv1_zeropadding')(img_input)
    x = Convolution2D(nb_filter, (7, 7), strides=(2, 2), name='conv1', use_bias=False)(x)
    x = BatchNormalization(epsilon=eps, axis=concat_axis, name='conv1_bn')(x)
    x = Scale(axis=concat_axis, name='conv1_scale')(x)
    x = Activation('relu', name='relu1')(x)
    x = ZeroPadding2D((1, 1), name='pool1_zeropadding')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2), name='pool1')(x)

    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        stage = block_idx + 2
        x, nb_filter = dense_block(x, stage, nb_layers[block_idx], nb_filter, growth_rate, dropout_rate=dropout_rate,
                                   weight_decay=weight_decay)

        # Add transition_block
        x = transition_block(x, stage, nb_filter, compression=compression, dropout_rate=dropout_rate,
                             weight_decay=weight_decay)
        nb_filter = int(nb_filter * compression)

    final_stage = stage + 1
    x, nb_filter = dense_block(x, final_stage, nb_layers[-1], nb_filter, growth_rate, dropout_rate=dropout_rate,
                               weight_decay=weight_decay)

    x = BatchNormalization(epsilon=eps, axis=concat_axis, name='conv' + str(final_stage) + '_blk_bn')(x)
    x = Scale(axis=concat_axis, name='conv' + str(final_stage) + '_blk_scale')(x)
    x = Activation('relu', name='relu' + str(final_stage) + '_blk')(x)
    x = GlobalAveragePooling2D(name='pool' + str(final_stage))(x)

    x = Dense(classes, name='fc6')(x)
    x = Activation('sigmoid', name='prob')(x)

    model = Model(inputs=[img_input_layer, label_input_layer], outputs=x, name='densenet')

    if weights_path is not None:
        model.load_weights(weights_path)

    return model


def conv_block(x, stage, branch, nb_filter, dropout_rate=None, weight_decay=1e-4):
    '''Apply BatchNorm, Relu, bottleneck 1x1 Conv2D, 3x3 Conv2D, and option dropout
        # Arguments
            x: input tensor
            stage: index for dense block
            branch: layer index within each dense block
            nb_filter: number of filters
            dropout_rate: dropout rate
            weight_decay: weight decay factor
    '''
    eps = 1.1e-5
    conv_name_base = 'conv' + str(stage) + '_' + str(branch)
    relu_name_base = 'relu' + str(stage) + '_' + str(branch)

    # 1x1 Convolution (Bottleneck layer)
    inter_channel = nb_filter * 4
    x = BatchNormalization(epsilon=eps, axis=concat_axis, name=conv_name_base + '_x1_bn')(x)
    x = Scale(axis=concat_axis, name=conv_name_base + '_x1_scale')(x)
    x = Activation('relu', name=relu_name_base + '_x1')(x)
    x = Convolution2D(inter_channel, (1, 1), name=conv_name_base + '_x1', use_bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    # 3x3 Convolution
    x = BatchNormalization(epsilon=eps, axis=concat_axis, name=conv_name_base + '_x2_bn')(x)
    x = Scale(axis=concat_axis, name=conv_name_base + '_x2_scale')(x)
    x = Activation('relu', name=relu_name_base + '_x2')(x)
    x = ZeroPadding2D((1, 1), name=conv_name_base + '_x2_zeropadding')(x)
    x = Convolution2D(nb_filter, (3, 3), name=conv_name_base + '_x2', use_bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    return x


def transition_block(x, stage, nb_filter, compression=1.0, dropout_rate=None, weight_decay=1E-4):
    ''' Apply BatchNorm, 1x1 Convolution, averagePooling, optional compression, dropout
        # Arguments
            x: input tensor
            stage: index for dense block
            nb_filter: number of filters
            compression: calculated as 1 - reduction. Reduces the number of feature maps in the transition block.
            dropout_rate: dropout rate
            weight_decay: weight decay factor
    '''

    eps = 1.1e-5
    conv_name_base = 'conv' + str(stage) + '_blk'
    relu_name_base = 'relu' + str(stage) + '_blk'
    pool_name_base = 'pool' + str(stage)

    x = BatchNormalization(epsilon=eps, axis=concat_axis, name=conv_name_base + '_bn')(x)
    x = Scale(axis=concat_axis, name=conv_name_base + '_scale')(x)
    x = Activation('relu', name=relu_name_base)(x)
    x = Convolution2D(int(nb_filter * compression), (1, 1), name=conv_name_base, use_bias=False)(x)

    if dropout_rate:
        x = Dropout(dropout_rate)(x)

    x = AveragePooling2D((2, 2), strides=(2, 2), name=pool_name_base)(x)

    return x


def dense_block(x, stage, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1e-4,
                grow_nb_filters=True):
    ''' Build a dense_block where the output of each conv_block is fed to subsequent ones
        # Arguments
            x: input tensor
            stage: index for dense block
            nb_layers: the number of layers of conv_block to append to the model.
            nb_filter: number of filters
            growth_rate: growth rate
            dropout_rate: dropout rate
            weight_decay: weight decay factor
            grow_nb_filters: flag to decide to allow number of filters to grow
    '''

    eps = 1.1e-5
    concat_feat = x

    for i in range(nb_layers):
        branch = i + 1
        x = conv_block(concat_feat, stage, branch, growth_rate, dropout_rate, weight_decay)
        concat_feat = Concatenate()([concat_feat, x])

        if grow_nb_filters:
            nb_filter += growth_rate

    return concat_feat, nb_filter

#Utils

In [4]:
import os

import numpy
import numpy as np
from keras.datasets.fashion_mnist import load_data
from keras_preprocessing.image import ImageDataGenerator
from numpy import expand_dims
from numpy.random import randint
from numpy.random import randn

image_size = 224
noise_size = 400
dataset_labels = 7
clip_value = 0.01

# Define datagen. Here we can define any transformations we want to apply to images
datagen = ImageDataGenerator()

# define training directory that contains subfolders
train_dir = os.path.join("gdrive", "My Drive", "Datasets", "HAM10000", "reorganized")


def load_real_data_old():
    # load dataset
    (trainX, trainy), (_, _) = load_data()
    # expand to 3d, e.g. add channels
    X = expand_dims(trainX, axis=-1)
    # convert from ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [-1,1]
    X = (X - 127.5) / 127.5
    return [X, trainy]


def load_real_data():
    # emulation dataset loading
    train_data_keras = datagen.flow_from_directory(directory=train_dir,
                                                   class_mode='categorical',
                                                   batch_size=1,  # 16 images at a time
                                                   target_size=(image_size, image_size),
                                                   color_mode='grayscale')  # Resize images
    # split into images and labels
    images, labels = next(train_data_keras)
    size = train_data_keras.samples
    images = numpy.zeros([size, images[0][0].size, images[0][0].size, 1])
    return [images, labels]


def get_images_and_labels(n_samples):
    train_data_keras = datagen.flow_from_directory(directory=train_dir,
                                                   class_mode='sparse',
                                                   batch_size=n_samples,  # 16 images at a time
                                                   target_size=(image_size, image_size),
                                                   color_mode='grayscale')  # Resize images
    # split into images and labels
    images, labels = next(train_data_keras)
    # labels = numpy.argmax(labels, axis=-1)
    # convert from ints to floats
    images = images.astype('float32')
    # scale from [0,255] to [-1,1]
    images = (images - 127.5) / 127.5
    # generate class labels
    return images, labels.astype(int)


def generate_real_samples(n_samples):
    images, labels = get_images_and_labels(n_samples)
    # generate class labels
    y = -ones((n_samples, 1))
    return [images, labels], y


def get_noise(latent_dim, n_samples):
    return randn(latent_dim * n_samples).reshape(n_samples, latent_dim)


# use the generator to generate n fake examples, with class labels
def generate_fake_samples(generator, latent_dim, n_samples):
    img_input, labels_input = get_images_and_labels(n_samples)
    z_input = get_noise(latent_dim, n_samples)
    # predict outputs
    images = generator.predict([img_input, labels_input, z_input])
    # create class labels
    y = ones((n_samples, 1))
    return [images, labels_input], y


def get_noise_data(batch_size):
    return np.random.normal(0, 0.001, size=[batch_size, noise_size]).astype(np.float32)


def save_plot(examples, epoch, n=10):
    # plot images
    for i in range(n * n):
        # define subplot
        pyplot.subplot(n, n, 1 + i)
        # turn off axis
        pyplot.axis('off')
        # plot raw pixel data
        pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
    # save plot to file
    filename = 'generated_plot_e%03d.png' % (epoch + 1)
    pyplot.savefig(filename)
    pyplot.close()


from enum import IntEnum


class FashionLabel(IntEnum):
    Tshirt = 0
    Trouser = 1
    Pullover = 2
    Dress = 3
    Coat = 4
    Sandal = 5
    Shirt = 6
    Sneaker = 7
    Bag = 8
    Ankle_boot = 9

class CancerLabel(IntEnum):
    akiec = 0
    bcc = 1
    bkl = 2
    df = 3
    mel = 4
    nv = 5
    vasc = 6

#Nets

In [5]:
import tensorflow as tf
from keras import backend
from keras.constraints import Constraint
from keras.layers import Input, Dense, Concatenate, ReLU, Conv2D, Conv2DTranspose, Reshape, BatchNormalization, \
    Activation, Embedding, LeakyReLU, Dropout
from keras.models import Model
from tensorflow import Tensor, keras

def relu_bn(inputs: Tensor) -> Tensor:
    relu = ReLU()(inputs)
    bn = BatchNormalization()(relu)
    return bn


drop_out_rate = 0.5
w_initializer = tf.keras.initializers.TruncatedNormal(mean=0., stddev=0.02)


def wasserstein_loss(y_true, y_pred):
    return backend.mean(y_true * y_pred)


# clip model weights to a given hypercube
class ClipConstraint(Constraint):
    # set clip value when initialized
    def __init__(self, clip_value):
        self.clip_value = clip_value

    # clip model weights to hypercube
    def __call__(self, weights):
        return backend.clip(weights, -self.clip_value, self.clip_value)

    # get the config
    def get_config(self):
        return {'clip_value': self.clip_value}


def up_scaling_layer(x, n_filters):
    kernel = 1
    stride = 2
    x = Conv2DTranspose(n_filters, kernel_size=kernel, strides=stride, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)
    x = Dropout(drop_out_rate)(x)
    return x


def down_scaling_layer(x, n_filters):
    kernel = 1
    stride = 2
    x = Conv2D(n_filters, kernel_size=kernel, strides=stride, padding='same')(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = BatchNormalization()(x)
    x = Dropout(drop_out_rate)(x)
    return x


def resNet_block(x, n_filters, scaling):
    kernel = 3
    stride = 1
    x = Conv2D(n_filters, kernel_size=kernel, strides=stride, padding='same')(x)
    x = Conv2D(n_filters, kernel_size=kernel, strides=stride, padding='same')(x)
    x = Conv2D(n_filters, kernel_size=kernel, strides=stride, padding='same')(x)
    x = Conv2D(n_filters, kernel_size=kernel, strides=stride, padding='same')(x)
    x = BatchNormalization()(x)
    if scaling == 'up':
        x = up_scaling_layer(x, n_filters)
    else:
        x = down_scaling_layer(x, n_filters)
    print("x shape: ", x.shape)
    return x


def representation_layer(layer, noise_input):
    filter = 3
    stride = 2
    kernel = 4

    dense = Dense(7 * 7 * filter, kernel_initializer=w_initializer)(noise_input)

    reshape = Reshape((7, 7, filter))(dense)

    noise_conv = Conv2DTranspose(kernel_size=kernel,
                                 strides=stride,
                                 filters=2 * filter,
                                 padding="same",
                                 kernel_initializer=w_initializer)(reshape)
    concat_layer = Concatenate()([layer, noise_conv])
    print("concat_layer shape: ", concat_layer.shape)
    return concat_layer


def img_n_label_layer(input_shape, n_classes):
    # label input
    label_input_layer = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 50)(label_input_layer)
    # scale up to image dimensions with linear activation
    n_nodes = input_shape[0] * input_shape[1]
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((input_shape[0], input_shape[1], 1))(li)
    # image input
    img_input_layer = Input(shape=input_shape)
    # concat label as a channel
    img_n_label_input_layer = Concatenate()([img_input_layer, li])
    return img_input_layer, img_n_label_input_layer, label_input_layer


def define_generator(latent_dim, input_shape, n_classes):
    n_filters = 3

    img_input_layer, img_n_label_input_layer, label_input_layer = img_n_label_layer(input_shape, n_classes)

    x1 = resNet_block(img_n_label_input_layer, n_filters, 'down')
    x2 = resNet_block(x1, 2 * n_filters, 'down')
    x3 = resNet_block(x2, 4 * n_filters, 'down')
    x4 = resNet_block(x3, 8 * n_filters, 'down')

    noise_input_layer = Input(shape=latent_dim)
    decoded_img_and_noise = representation_layer(x4, noise_input_layer)

    x5 = resNet_block(decoded_img_and_noise, 8 * n_filters, 'up')
    skip_connection_3_5 = Concatenate(axis=-1)([x3, x5])
    x6 = resNet_block(skip_connection_3_5, 4 * n_filters, 'up')
    skip_connection_2_6 = Concatenate(axis=-1)([x2, x6])
    x7 = resNet_block(skip_connection_2_6, 2 * n_filters, 'up')
    skip_connection_1_7 = Concatenate(axis=-1)([x1, x7])
    x8 = resNet_block(skip_connection_1_7, 1, 'up')

    gen_output = Activation('tanh')(x8)

    model = Model(inputs=[img_input_layer, label_input_layer, noise_input_layer], outputs=gen_output)
    return model


def define_discriminator(input_shape, n_classes):
    img_input_layer, img_n_label_input_layer, label_input_layer = img_n_label_layer(input_shape, n_classes)

    model = DenseNet(img_input_layer, img_n_label_input_layer, label_input_layer, reduction=0.5, classes=1)

    opt = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.99)
    model.compile(optimizer=opt, loss=wasserstein_loss, metrics=['accuracy'])
    return model


# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
    # d_model.trainable = False
    # get img, label and noise inputs from generator model
    gen_img, gen_label, gen_noise = g_model.input
    # get image output from the generator model
    gen_output = g_model.output
    # connect image output and label input from generator as inputs to discriminator
    gan_output = d_model([gen_output, gen_label])
    # define gan model as taking img, noise and label and outputting a classification
    model = Model([gen_img, gen_label, gen_noise], gan_output)
    # compile model
    opt = keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.99)
    model.compile(loss=wasserstein_loss, optimizer=opt)
    return model

#Train

In [ ]:
# example of training an conditional gan on the fashion mnist dataset
from matplotlib import pyplot
from numpy import ones, mean
import tensorflow as tf
from matplotlib import pyplot as plt


# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist):
    # plot history
    pyplot.plot(d1_hist, label='crit_real')
    pyplot.plot(d2_hist, label='crit_fake')
    pyplot.plot(g_hist, label='gen')
    pyplot.legend()
    pyplot.savefig('plot_line_plot_loss.png')
    pyplot.close()


# train the generator and discriminator
def train(g_model, d_model, gan_model, latent_dim, n_epochs=100, n_batch=128):
    bat_per_epo = int(dataset[0].shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    n_critic = 5
    # lists for keeping track of loss
    c1_hist, c2_hist, g_hist = list(), list(), list()
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    X_real, labels_real, y_real = [], [], []
    for i in range(n_steps):
        # update the critic more than the generator
        c1_tmp, c2_tmp = list(), list()
        for _ in range(n_critic):
            # get randomly selected 'real' samples
            [X_real, labels_real], y_real = generate_real_samples(half_batch)
            # update critic model weights
            c_loss1 = d_model.train_on_batch([X_real, labels_real], y_real)
            c1_tmp.append(c_loss1)
            # generate 'fake' examples
            [X_fake, labels_fake], y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
            # update critic model weights
            c_loss2 = d_model.train_on_batch([X_fake, labels_fake], y_fake)
            c2_tmp.append(c_loss2)
            # store critic loss
            c1_hist.append(mean(c1_tmp))
            c2_hist.append(mean(c2_tmp))

            for l in d_model.layers:
                weights = l.get_weights()
                print("Unclipped weights ", weights)
                weights = [tf.clip_by_value(w, -clip_value, clip_value) for w in weights]
                l.set_weights(weights)
                print("Clipped weights ", l.get_weights)
                
        # prepare points in latent space as input for the generator
        img_input, labels_input, y_gan = X_real, labels_real, y_real
        # img_input, labels_input, y_gan = Utils.generate_real_samples(n_batch)
        z_input = get_noise(latent_dim, half_batch)
        # update the generator via the critic's error
        g_loss = gan_model.train_on_batch([img_input, labels_input, z_input], y_gan)
        g_hist.append(g_loss)
        # summarize loss on this batch
        print('>%d/%d, c1=%.3f, c2=%.3f g=%.3f' % (i + 1, n_steps, c1_hist[-1], c2_hist[-1], g_loss))
        # evaluate the model performance every 'epoch'
        if (i + 1) % bat_per_epo == 0:
            summarize_performance(i, g_model, d_model, dataset)
    # line plots of loss
    plot_history(c1_hist, c2_hist, g_hist)


# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, n_samples=15):
    # prepare real samples
    # X_real, y_real = generate_real_samples(dataset, n_samples)
    # # evaluate discriminator on real examples
    # _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # # prepare fake examples
    # x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # # evaluate discriminator on fake examples
    # _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # # summarize discriminator performance
    # print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real * 100, acc_fake * 100))
    # save plot
    # save_plot(x_fake, epoch)
    # save the generator model tile file
    filename = 'dis_model_%03d.h5' % (epoch + 1)
    d_model.save(filename)
    filename = 'gen_model_%03d.h5' % (epoch + 1)
    g_model.save(filename)


# size of the latent space
latent_dim = 100
# load image data
dataset = load_real_data()
image_shape = dataset[0][0].shape
# create the discriminator
d_model = define_discriminator(image_shape, dataset_labels)
# create the generator
g_model = define_generator(latent_dim, image_shape, dataset_labels)
# create the gan
gan_model = define_gan(g_model, d_model)
# train model
train(g_model, d_model, gan_model, latent_dim)

Found 10015 images belonging to 7 classes.
